# 신문기사 데이터 추출 및 CSV 저장
이 노트북에서는 GDELT API를 사용하여 특정 키워드에 맞는 신문기사를 추출하고, Newspaper3k 라이브러리를 사용해 각 기사의 본문을 파싱한 후, 이를 CSV 파일로 저장하는 방법을 설명합니다.

### 1. 필요한 라이브러리 설치 및 임포트

이 단계에서는 필요한 라이브러리들을 설치합니다. gdeltdoc는 GDELT API를 사용하여 기사를 검색할 때 필요하고, newspaper3k는 각 기사에서 본문을 추출할 때 사용됩니다. 또한, pandas 라이브러리를 사용해 데이터를 쉽게 다룰 수 있습니다.

In [1]:
!pip install gdeltdoc
!pip install newspaper3k==0.2.8
!pip install lxml_html_clean
!pip install pandas

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/7.4 MB ? eta -:--:--
     ---- ----------------------------------- 0.8/7.4 MB 4.2 MB/s eta 0:00:02
     ----------- ---------------------------- 2.1/7.4 MB 5.3 MB/s eta 0:00:01
     ------------------ --------------------- 3.4/7.4 MB 5.6 MB/s eta 0:00:01
     ------------------------ --------------- 4.5/7.4 MB 5.4 MB/s eta 0:00:01
     ------------------------------- -------- 5.8/7.4 MB 5.5 MB/s eta 0:00:01
     ------------------------------------ --- 6.8/7.4 MB 5.4 MB/s eta 0:00:01
     ---------------------------------------- 7.4/7.4 MB 5.2 MB/s  0:00:01
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finish

  DEPRECATION: Building 'tinysegmenter' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'tinysegmenter'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'feedfinder2' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'feedfinder2'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'jieba3k' using the legacy setup.py bdis

### 2. 라이브러리 임포트
필요한 라이브러리를 코드에 임포트합니다. 

In [2]:
from gdeltdoc import GdeltDoc, Filters
from newspaper import Article
import pandas as pd

### 3. GDELT API 필터 설정

GDELT API를 사용하여 원하는 기사를 추출하기 위한 필터를 설정합니다. Filters 클래스를 사용하여 날짜, 키워드, 도메인 등 다양한 조건을 지정할 수 있습니다.

In [3]:
f = Filters(
    keyword = "Microsoft",  # 키워드: Microsoft 관련 기사만 추출
    start_date = "2024-05-01",  # 시작 날짜
    end_date = "2024-05-25",  # 종료 날짜
    num_records=10,  # 가져올 기사의 수
    domain ="nytimes.com",  # 특정 도메인(여기서는 NYTimes)
    country="US",  # 국가 설정 (US)
)

### 4. GDELT 데이터 검색

설정한 필터에 맞는 기사를 GDELT API를 통해 검색하여 결과를 pandas DataFrame으로 변환합니다.

In [4]:
# GDELT 객체 생성
gd = GdeltDoc()

# 필터에 맞는 기사 검색
articles = gd.article_search(f)

# 기사의 URL과 제목을 포함한 DataFrame 생성
articles_data = []
for index, row in articles.iterrows():
    url = row['url']
    title = row['title']
    
    # Newspaper3k 라이브러리를 사용하여 기사 본문 추출
    article = Article(url)
    article.download()
    article.parse()
    text = article.text  # 기사 본문
    
    # DataFrame에 추가
    articles_data.append({
        "title": title,
        "url": url,
        "text": text
    })

# DataFrame 생성
articles_df = pd.DataFrame(articles_data)

# 결과 출력
articles_df.head()  # 첫 5개의 기사 출력

,title,url,text
0,The C . E . O . s Who Just Wont Quit - The New...,https://www.nytimes.com/2024/05/09/magazine/fo...,Of the many riddles that confront corporate ch...
1,"C . Gordon Bell , Creator of a Personal Comput...",https://www.nytimes.com/2024/05/21/technology/...,"C. Gordon Bell, a technology visionary whose c..."
2,"Wayve , an A . I . Start - Up for Autonomous D...",https://www.nytimes.com/2024/05/06/technology/...,"Wayve, a London maker of artificial intelligen..."
3,Biden to Announce A . I . Center in Wisconsin ...,https://www.nytimes.com/2024/05/08/us/politics...,President Biden on Wednesday announced the cre...
4,How Rich Candidates Burned Cash on Running for...,https://www.nytimes.com/2024/05/16/us/politics...,The costly realm of campaign politics has clai...


### 5. CSV 파일로 저장

최종적으로 추출한 기사 데이터를 pandas DataFrame으로 저장한 후, 이를 CSV 파일로 저장합니다.

In [5]:
# CSV 파일로 저장
articles_df.to_csv('articles_data.csv', index=False)